In [1]:
#Allows relative imports
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
#imports 
from src.preprocessing import *
from src.models import deepcat_cnn
from src.torch_util import *
from src.dataloader import *
from src.train_eval_helpers import *
%load_ext autoreload
%autoreload 2
#checking gpu status
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import BatchSampler, RandomSampler    
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
import math

import matplotlib as mpl
mpl.rcParams['figure.dpi']= 150
import seaborn as sns
sns.set_style('darkgrid')
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using : {}".format(device))
else:
    device = torch.device('cpu')
    print("Using : {}".format(device))
    
RANGE = range(12,17)
TRAINDIR = '../TrainingData/'
files = os.listdir(TRAINDIR)
files.remove('readme.md')
#Reading data
train_normal = read_seq(TRAINDIR+files[0])
train_tumor = read_seq(TRAINDIR+files[2])

Using : cuda


In [ ]:
def PredictCancer(f,dir_prefix):
    ## f: input iSMART result file
    ## N: top N most frequent CDR3s will be included in the analysis
    gf=open(f)
    CDR3s=[]
    for ll in gf.readlines():
        cc=ll.strip().split('\t')[0]
        if not cc.startswith('C') or not cc.endswith('F'):
            continue
        CDR3s.append(cc)
    CDR3sDict={}
    for cc in CDR3s:
        if len(pat.findall(cc))>0:
            continue
        ll=len(cc)
        ccF=AAindexEncoding(cc)
        if ll not in CDR3sDict:
            CDR3sDict[ll]=[ccF]
        else:
            CDR3sDict[ll].append(ccF)
    ScoreDict={}
    XX=[]
    for LL in range(12,17):
        CDR3_classifier=tf.estimator.Estimator(model_fn=ModelDict[LL],model_dir=dir_prefix+'/CDR3_classifier_PCA_LL'+str(LL)+'_L2_k2f8d10_tCi01'+'/')
        if LL in CDR3sDict:
            eval_input_fn=tf.estimator.inputs.numpy_input_fn(
                x={'x':np.array(CDR3sDict[LL])},        
                num_epochs=1,
                shuffle=False)
        else:
            continue
        eval_results=CDR3_classifier.predict(input_fn=eval_input_fn)
        xx=[]
        for x in eval_results:
            xx.append(x['probabilities'][1])
        ScoreDict[LL]=xx
        XX+=xx
    mms=[]
    for kk in ScoreDict:
        mms.append((kk,np.mean(ScoreDict[kk])))
    CancerScore=np.mean(XX)
    return CancerScore,XX

#    return mms, XX, ScoreDict, CancerScore

def PredictBatch(DIR, dir_prefix=curPath+'/tmp/'):
    ffs=os.listdir(DIR)
    mmsList=[]
    SDList=[]
    XXList=[]
    for ff in ffs:
        mms, XX, SD=PredictCancer(DIR+ff, dir_prefix=dir_prefix)
        mmsList.append(mms)
        SDList.append(SD)
        XXList.append(XX)
    return ffs, mmsList, SDList, XXList
    
if len(sys.argv) > 1:   
 DIR=sys.argv[1]
 DIR1=os.path.basename(DIR)
 ffs=os.listdir(DIR)
 dir_prefix=sys.argv[2]
 CC=[]
 ffss=[]
 for ff in ffs:
   if ff == 'README.md':
     continue
   else: 
     score,XX1 = PredictCancer(DIR+'/'+ff, dir_prefix+'/tmp/')
     CC.append(score)
     ffss.append(ff)  
 CC=np.array(CC)
 ffss=np.array(ffss)
 if sys.argv[3] == '-t':
   with open('Cancer_score_'+DIR1+'.txt', 'w') as f:
      writer = csv.writer(f, delimiter='\t')
      writer.writerows(zip(ffss,CC))  
 elif sys.argv[3] == '-r':       
   with open('Cancer_score.txt', 'w') as f:
      writer = csv.writer(f, delimiter='\t')
      writer.writerows(zip(ffss,CC))  

In [2]:
def read_seq_ismart(filename):
    """
    Read sequences from a txt, and returns an array of the sequences.
    Using numpy arrays because of easier fancy indexing when generating data from sequences
    """
    if '.txt' not in filename:
        print("Non .txt file given, exiting")
        return

    data = []
    with open(filename, 'r') as f:
        for line in f.readlines():
            seq = line.strip()
            if not seq.startswith('C') or not seq.endswith('F'):continue
            data.append(seq)
    return np.array(data)

def aaindex_encoding(seq, device):
    """Encodes the AA indices to a given sequence"""
    n_aa = len(seq)
    temp = np.zeros([n_aa, n_feats], dtype=np.float32)
    for idx in range(n_aa):
        aa = seq[idx]
        temp[idx] = AAidx_Dict[aa]
    temp = np.transpose(temp)
    aa_encoding = torch.from_numpy(temp)
    aa_encoding = aa_encoding.unsqueeze(0)
    if device == torch.device('cuda'):
        aa_encoding = aa_encoding.to(device)
    return aa_encoding

In [30]:
import pandas as pd

path = '../SampleData/Cancer/'
print(os.listdir(path)[0:5])
files = [x for x in os.listdir(path) if x.endswith('.txt')]
data = pd.read_csv(os.path.join(path,files[0]), sep='\t', header = 0)
display(data.head())
data['aminoAcid'].values[0:10]

['README.md', 'TestReal-BR01B.tsv_ClusteredCDR3s_7.5.txt', 'TestReal-BR05B.tsv_ClusteredCDR3s_7.5.txt', 'TestReal-BR07B.tsv_ClusteredCDR3s_7.5.txt', 'TestReal-BR13B.tsv_ClusteredCDR3s_7.5.txt']


,aminoAcid,vMaxResolved,frequencyCount....,Group
0,CASSLELGALGGNTIYF,TCRBV07-09,0.000133,1
1,CASSLELGALGGNTIYF,TCRBV07-09,0.000063,1
2,CASGTSGFTDTQYF,TCRBV30-01*01,0.000008,2
3,CAWRTSGLTDTQYF,TCRBV30-01*01,0.000008,2
4,CSVGGGGFNEKLFF,TCRBV29-01*01,0.000016,3


array(['CASSLELGALGGNTIYF', 'CASSLELGALGGNTIYF', 'CASGTSGFTDTQYF',
       'CAWRTSGLTDTQYF', 'CSVGGGGFNEKLFF', 'CSVVGGGFNEKLFF',
       'CASSLVSANYGYTF', 'CASSLVSANYGYTF', 'CASSSRGGYTEAFF',
       'CASSSRGGYTEAFF'], dtype=object)

In [74]:
mask = data['aminoAcid'].str.startswith('C') & data['aminoAcid'].str.endswith('F')
data[mask]['aminoAcid'].values

array(['CASSLELGALGGNTIYF', 'CASSLELGALGGNTIYF', 'CASGTSGFTDTQYF',
       'CAWRTSGLTDTQYF', 'CSVGGGGFNEKLFF', 'CSVVGGGFNEKLFF',
       'CASSLVSANYGYTF', 'CASSLVSANYGYTF', 'CASSSRGGYTEAFF',
       'CASSSRGGYTEAFF', 'CASSLNLASYEQYF', 'CASSLGLASYEQYF',
       'CASSETASGNTIYF', 'CASSETASGNTIYF', 'CASSLGQGGYEQYF',
       'CASSLSQGGYEQYF', 'CASSPTGGNQPQHF', 'CASSPSGGNQPQHF',
       'CAWAGTSNTGELFF', 'CACRGTGNTGELFF', 'CASSPGQGVYEQYF',
       'CASSPGQGAYEQYF', 'CASSLSRSNQPQHF', 'CASSLSRSNQPQHF',
       'CASSLQGAMETQYF', 'CASSLQGAQETQYF', 'CASSLQGRNQPQHF',
       'CASSLRGRNQPQHF', 'CSVWDRGGNQPQHF', 'CSAWDRAGNQPQHF',
       'CASSLSRSNQPQHF', 'CASSLDRSNQPQHF', 'CASEVGGSNQPQHF',
       'CASSLGGSNQPQHF', 'CASSLSAGGYEQYF', 'CASSLSAGGYEQYF',
       'CSVVRGIGTDTQYF', 'CSVERGMGTDTQYF', 'CASSVEPDSYEQYF',
       'CASSVEPSSYEQYF', 'CASSPDNSNQPQHF', 'CASSPDSSNQPQHF',
       'CSVGSGGTNEKLFF', 'CSVGTGGTNEKLFF', 'CSVGTGNTNEKLFF',
       'CSVEKQINTGELFF', 'CSVEKQLNTGELFF', 'CSALRDRSSYEQYF',
       'CSVKRDRDSY

### to read data

In [39]:
os.path.basename(os.path.join(path,files[0]))

'TestReal-BR01B.tsv_ClusteredCDR3s_7.5.txt'

In [ ]:
files = [x for x in os.listdir(path) if x.endswith('.txt')]

for f in files:
    data = pd.read_csv(os.path.join(path,files[0]), sep='\t', header = 0)
    #seqs = data['aminoAcid'] #An array containing sequences as str
    mask = data['aminoAcid'].str.startswith('C') & data['aminoAcid'].str.endswith('F')
    seqs = data[mask]['aminoAcid'].copy()

In [170]:
def read_ismart(filename):
    """
    Reads a .txt file, assumes that it is in the format outputed by iSMARTm.py
    Returns a numpy array to be used for aaindex_encoding()
    """
    data = pd.read_csv(filename, sep='\t', header=0)
    data['len'] = data.apply(lambda x: len(x['aminoAcid']),axis=1)
    data = data.query('len>=12 & len<=16').copy()
    mask = data['aminoAcid'].str.startswith('C') & data['aminoAcid'].str.endswith('F')
    data = data[mask].sort_values('len', ascending=True).copy()
    seqs = data['aminoAcid'].copy().values
    return seqs, data

def generate_features(sequences, keys = range(12,17), device = None, shuffle=False):
    """
    Takes a np.array, returns a dictionnary.
    Behaves like generate_features_labels except that it doesn't have a label.
    """
    seqlens = np.array([len(seqs) for seqs in sequences])
    print("Getting data")
    feature_dict = {}
    #Only keep sequences with length 12 to 16
    for length in keys:
        #Using numpy to create mask for fancy indexing, converting to tensors later
        
        mask = np.where(seqlens==length)[0]
        #Reusing the code from DeepCAT for Labels
        
        data = []
        for seq in sequences[mask]:
            if len(PAT.findall(seqs))>0:continue #Skipping a sequence if it matches an unwanted CDR3 pattern 
            data.append(aaindex_encoding(seq, 'cpu'))

        data = torch.stack(data) #Stack a list of tensors into a single tensor

        #Sends to cuda. Shouldn't do this in batch-train because every tensors will be on GPU
        #leading to out of memory issues

        feature_dict[length] = data
        del data
        
    print("Data device =",feature_dict[12].device)
    print("Done loading, returning features")
    return feature_dict

def aaindex_encoding(seq, device='cuda'):
    """Encodes the AA indices to a given sequence"""
    n_aa = len(seq)
    temp = np.zeros([n_aa, n_feats], dtype=np.float32)
    for idx in range(n_aa):
        aa = seq[idx]
        temp[idx] = AAidx_Dict[aa]
    temp = np.transpose(temp)
    aa_encoding = torch.from_numpy(temp)
    aa_encoding = aa_encoding.unsqueeze(0)
    if device == torch.device('cuda'):
        aa_encoding = aa_encoding.to(device)
    return aa_encoding

def get_data_df(filename):
    _, df = read_ismart(filename)
    df['len'] = df.apply(lambda x: len(x['aminoAcid']),axis=1)
    df = df.query('len>=12 & len<=16').copy()
    df['features'] = df['aminoAcid'].apply(aaindex_encoding)
    return df

In [ ]:
def generate_features(sequences, keys = range(12,17), device = None, shuffle=False):
    """
    Takes a np.array, returns a dictionnary.
    Behaves like generate_features_labels except that it doesn't have a label.
    """
    seqlens = np.array([len(seqs) for seqs in sequences])
    print("Getting data")
    feature_dict = {}
    #Only keep sequences with length 12 to 16
    for length in keys:
        #Using numpy to create mask for fancy indexing, converting to tensors later
        
        mask = np.where(seqlens==length)[0]
        #Reusing the code from DeepCAT for Labels
        
        data = []
        for seq in sequences[mask]:
            if len(PAT.findall(seqs))>0:continue #Skipping a sequence if it matches an unwanted CDR3 pattern 
            data.append(aaindex_encoding(seq, 'cpu'))

        data = torch.stack(data) #Stack a list of tensors into a single tensor

        #Sends to cuda. Shouldn't do this in batch-train because every tensors will be on GPU
        #leading to out of memory issues

        feature_dict[length] = data
        del data

In [177]:
feats = []
for seq in df.query('len==12')['aminoAcid'].values:
    feats.append(aaindex_encoding(seq))
feats = torch.stack(feats).to('cuda')

In [179]:
feats.shape

torch.Size([47, 1, 15, 12])

In [224]:
def get_feats_tensor(sequences, device):
    "Sequences : numpy array"
    feats = []
    for seq in sequences:
        feats.append(aaindex_encoding(seq))
    feats = torch.stack(feats).to('cuda')
    return feats

In [237]:
#Here for a single file
models = load_models([12,13,14,15,16],'../output/testplots/')
#set all models to eval
for k in keys:
    models[k].eval()
    models[k].to('cuda')
    
_, df = read_ismart(os.path.join(path,files[0])) #sorts the df by Length

df['prob_cancer']=None
for l in [12,13,14,15,16]:
    seqs=df.query('len==@l')['aminoAcid'].values
    feats = get_feats_tensor(seqs, device='cuda')
    logit, predictions, probs = models[l](feats)
    df.loc[df['len']==12,'prob_cancer'] = probs.detach().cpu()[:,1]

Models & weights loaded


In [247]:
def stuff(filename):
    _, df = read_ismart(filename) #sorts the df by Length
    df['prob_cancer']=None
    for l in [12,13,14,15,16]:
        seqs=df.query('len==@l')['aminoAcid'].values
        feats = get_feats_tensor(seqs, device='cuda')
        logit, predictions, probs = models[l](feats)
        df.loc[df['len']==l,'prob_cancer'] = probs.detach().cpu()[:,1]
    return df


,aminoAcid,vMaxResolved,frequencyCount....,Group,len,prob_cancer
170,CASSLENQPQHF,TCRBV07-02*01,0.001300,81,12,0.007298
224,CASSGSSYGYTF,TCRBV06-01*01,0.000016,105,12,0.015095
223,CASSGTSYGYTF,TCRBV06-01*01,0.000039,105,12,0.023348
222,CASSTGVYGYTF,TCRBV19-01,0.000039,104,12,0.165869
221,CASSTGVYGYTF,TCRBV19-01,0.000016,104,12,0.165869
...,...,...,...,...,...,...
158,CASSVGVAGNTGELFF,TCRBV09-01,0.000016,76,16,0.453133
159,CASSVGLAGNTGELFF,TCRBV09-01,0.000016,76,16,0.223772
161,CASSQDTGSGANVLTF,TCRBV04-03*01,0.000016,77,16,0.020113
163,CASSLEYGSRPYEQYF,TCRBV07-02*01,0.000016,78,16,0.453133


In [312]:
path

'../SampleData/Cancer/'

In [327]:
for x in os.listdir('../SampleData/'):
    print(os.path.join('../SampleData/',x))

../SampleData/Cancer
../SampleData/Control


In [322]:
path, os.path.dirname(path)

('../SampleData/Cancer/', '../SampleData/Cancer')

In [329]:
for subfolder in os.listdir('../SampleData/'): #For each folder in in_directory
    subfolderpath = os.path.join('../SampleData', subfolder)#Get path to subfolder
    files = [os.path.join(subfolderpath,x) for x in os.listdir(subfolderpath) if x.endswith('.txt')] #Ge

In [337]:
scores = range(0,30)
MAPPING = {os.path.basename(file):cancer_score for (file,cancer_score) in zip(files,scores)}

In [340]:
os.path.basename(subfolderpath)

'Control'

In [323]:
os.path.basename(os.path.dirname(path))#(os.path.join(path,files[4]))

'Cancer'

In [263]:
cancer = stuff(os.path.join(path,files[4]))
non = stuff('../SampleData/Control/TestReal-HIP09029.tsv_ClusteredCDR3s_7.5.txt')
print(np.mean(cancer['prob_cancer']))
print(np.mean(non['prob_cancer']))

0.3295363025198227
0.2181643886145523


In [ ]:
with open('Cancer_score_'+DIR1+'.txt', 'w') as f:
      writer = csv.writer(f, delimiter='\t')
      writer.writerows(zip(ffss,CC))  
 elif sys.argv[3] == '-r':       
   with open('Cancer_score.txt', 'w') as f:
      writer = csv.writer(f, delimiter='\t')
      writer.writerows(zip(ffss,CC))  